In [1]:
import sys
print(sys.executable)

/scratch/go76fil/Programs/Python/Paper_Topic_Modelling/.venv/bin/python


In [3]:
import spacy
import numpy as np
import random
seed = 42
random.seed(seed)
np.random.seed(seed)

In [4]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [5]:
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 3000000

input_dir = "./data"

In [6]:
from spacy.lang.en.stop_words import STOP_WORDS
scientific_common_words = {
    "figure", "fig", "table", "et", "al", "etc", "eg", "i.e", "ie", 
    "eq", "conclusion", "introduction", "methods", "study", 
    "research", "author", "authors", "paper", "work", 
    "proposed", "presented", "based", "using", 
    "performed", "obtained", "observed", "used", "shown", "reported", 
    "significant", "important", "novel", "investigation", "algorithm", 
    "algorithms", "technique", "techniques", "parameters", 
    "parameter", "performance", "implemented", "implementation", "similar", 
    "different", "result", "respectively", "compare", "compared", 
    "comparison", "additional", "respectively", " ", "v.", "supra",
    "give", "follow", "know", "high","year", "term", "_",
    "find", "include", "tell" "try", "51ac", "pmpa", "id._note", "item", "coef",
    "interview", "percent", "go", "construct", "scale","000s", "ing", "pricewaterhousecooper",
    "analyse", "dmu?", "qsr?", "consequently", "furthermore", "atmo"
}
STOP_WORDS.update(scientific_common_words)
for word in scientific_common_words:
    print(f"Is '{word}' a stopword? {nlp.vocab[word].is_stop}")

Is 'observed' a stopword? True
Is 'high' a stopword? True
Is 'shown' a stopword? True
Is 'proposed' a stopword? True
Is 'construct' a stopword? True
Is 'go' a stopword? True
Is 'parameters' a stopword? True
Is 'implemented' a stopword? True
Is 'implementation' a stopword? True
Is 'introduction' a stopword? True
Is 'include' a stopword? True
Is 'know' a stopword? True
Is 'atmo' a stopword? True
Is 'techniques' a stopword? True
Is '000s' a stopword? True
Is 'coef' a stopword? True
Is 'algorithm' a stopword? True
Is 'scale' a stopword? True
Is 'authors' a stopword? True
Is 'significant' a stopword? True
Is 'author' a stopword? True
Is 'interview' a stopword? True
Is 'study' a stopword? True
Is ' ' a stopword? False
Is 'comparison' a stopword? True
Is 'etc' a stopword? True
Is 'compared' a stopword? True
Is 'different' a stopword? True
Is 'used' a stopword? True
Is 'result' a stopword? True
Is 'parameter' a stopword? True
Is 'paper' a stopword? True
Is 'et' a stopword? True
Is 'methods' a 

In [7]:
import os
folders = os.listdir(input_dir)
folders.sort()
folders = [folder for folder in folders if os.path.isdir(os.path.join(input_dir, folder))]
for folder in folders:
    print(folder)

Abstracts 1986-2005
All Papers
ISOF 2003 Papers
ISOF 2004 Papers
ISOF 2005 Papers
ISOF 2006 Papers
ISOF 2007 Papers
ISOF 2008 Papers
ISOF 2009 Papers
ISOF 2010 Papers
ISOF 2011 Papers
ISOF 2012 Papers
ISOF 2013 Papers
ISOF 2014 Papers
ISOF 2015 Papers
ISOF 2016 Papers
ISOF 2017 Papers
ISOF 2018 Papers
ISOF 2019 Papers
ISOF 2021 Papers
ISOF 2022 Papers
ISOF 2023 Papers
ISOF 2024 Papers


In [8]:
import re
folder_index = 1
try:
    year = re.findall(r'\d+', folders[folder_index])[0]
except IndexError:
    year = "All"
print(year)
print(folders[folder_index])

All
All Papers


In [9]:
from src.pdf_reader import MultiReader
from src.config import ReaderConfig

directory = os.path.join(input_dir,folders[folder_index])
config = ReaderConfig()
reader = MultiReader(directory,config)
pdf_texts = reader.read_all()

INFO:WFM.Config:Config Initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.MultiReader:Successfully read ./data/All Papers/10_Beheler_Norton_Sen_2005.pdf
INFO:WFM.MultiReader:Successfully read ./data/All Papers/10_2007_Watson_Stanworth_Purdy_Healeas_Holden.pdf
INFO:WFM.MultiReader:Successfully read ./data/All Papers/10_Ghedira_Perrigot_2004.pdf
INFO:WFM.PdfReader:PdfReader initialized
INFO:WFM.MultiReader:Successfully read ./data/All Papers/10_ISoF 2009_Dant_Weaven_Lapuka_Baker_Jeon.pdf
INFO:WFM.MultiReader:Successfully read ./data/A

In [15]:
unique_words = set()
for text in pdf_texts:
    words = text.split()  # Splits on whitespace
    unique_words.update(words)

print(f"Total Number of unique words: {len(unique_words)}")

Total Number of unique words: 186738


In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed
with ThreadPoolExecutor() as executor:
    doc_objects = [None] * len(pdf_texts)
    futures = []
    for index, text in enumerate(pdf_texts):
        logger.debug(f"Reading Text: {index}")
        futures.append(executor.submit(nlp, text))

    for future in as_completed(futures):
        index = futures.index(future) 
        print(f"Completed: {index}")
        doc_objects[index] = future.result()

Completed: 10
Completed: 2
Completed: 11
Completed: 9
Completed: 8
Completed: 1
Completed: 7
Completed: 4
Completed: 6
Completed: 5
Completed: 3
Completed: 16
Completed: 12
Completed: 17
Completed: 19
Completed: 13
Completed: 0
Completed: 18
Completed: 14
Completed: 20
Completed: 25
Completed: 29
Completed: 15
Completed: 30
Completed: 27
Completed: 23
Completed: 24
Completed: 28
Completed: 22
Completed: 26
Completed: 21
Completed: 35
Completed: 32
Completed: 43
Completed: 33
Completed: 37
Completed: 31
Completed: 34
Completed: 38
Completed: 40
Completed: 41
Completed: 46
Completed: 44
Completed: 36
Completed: 48
Completed: 47
Completed: 49
Completed: 53
Completed: 51
Completed: 58
Completed: 42
Completed: 50
Completed: 56
Completed: 54
Completed: 39
Completed: 45
Completed: 52
Completed: 57
Completed: 62
Completed: 61
Completed: 71
Completed: 60
Completed: 70
Completed: 59
Completed: 64
Completed: 65
Completed: 69
Completed: 55
Completed: 66
Completed: 67
Completed: 73
Completed: 68
Co

In [12]:
pattern = r'\b(?![A-Za-z]+\b)\S+\b'
def filter_tokens(token):
    return (
        not token.is_stop                    
        and not token.is_punct               
        and not token.like_email       
        and not token.like_url        
        and not token.like_num          
        and not token.ent_type_ == "PERSON"
        and token.pos_ != "PROPN"       
        and len(token.text) > 3      
        and not re.match(pattern, token.text)
    )
def process_doc(index, doc):
    """
    Apply the filter_tokens function to a single Doc object.
    """
    return index, [token.lemma_.lower() for token in doc if filter_tokens(token) and token.lemma_.strip()]

filtered_tokens = [None] * len(doc_objects)
with ThreadPoolExecutor() as executor:
    futures = []
    for index, doc in enumerate(doc_objects):
        futures.append(executor.submit(process_doc, index, doc))

    for future in as_completed(futures):
        index, tokens = future.result() 
        print(f"Filtered: {index}")
        filtered_tokens[index] = tokens 
print(len(filtered_tokens))
print(len(filtered_tokens[0]))

Filtered: 14
Filtered: 10
Filtered: 6
Filtered: 1
Filtered: 15
Filtered: 21
Filtered: 2
Filtered: 0
Filtered: 12
Filtered: 8
Filtered: 16
Filtered: 19
Filtered: 5
Filtered: 7
Filtered: 4
Filtered: 9
Filtered: 3
Filtered: 13
Filtered: 11
Filtered: 18
Filtered: 17
Filtered: 20
Filtered: 24
Filtered: 27
Filtered: 23
Filtered: 26
Filtered: 22
Filtered: 29
Filtered: 30
Filtered: 32
Filtered: 33
Filtered: 37
Filtered: 38
Filtered: 25
Filtered: 31
Filtered: 43
Filtered: 28
Filtered: 48
Filtered: 47
Filtered: 35
Filtered: 41
Filtered: 52
Filtered: 49
Filtered: 45
Filtered: 53
Filtered: 34
Filtered: 56
Filtered: 57
Filtered: 58
Filtered: 55
Filtered: 59
Filtered: 61
Filtered: 62
Filtered: 64
Filtered: 63
Filtered: 65
Filtered: 66
Filtered: 67
Filtered: 44
Filtered: 69
Filtered: 71
Filtered: 68
Filtered: 46
Filtered: 39
Filtered: 73
Filtered: 76
Filtered: 75
Filtered: 78
Filtered: 77
Filtered: 80
Filtered: 79
Filtered: 81
Filtered: 72
Filtered: 50
Filtered: 40
Filtered: 54
Filtered: 60
Filtered:

In [13]:
unique_tokens = set()
for tokens in filtered_tokens:
    unique_tokens.update(tokens)
print(f"Number of unique tokens after 1st stage of processing: {len(unique_tokens)}")

Number of unique tokens after 1st stage of processing: 25304


In [20]:
import pickle

with open('./output/store/filtered_tokens.pkl', 'wb') as f:
    pickle.dump((filtered_tokens,year), f)